In [6]:

import os
script_dir = os.path.dirname('Intermittent with DC Drive.ipynb')
results_dir = os.path.join(script_dir, 'Only PI_Results/')

if not os.path.isdir(results_dir):
    os.makedirs(results_dir)

In [9]:
from brian2 import *
%matplotlib inline
import numpy as np
from numpy import heaviside
from math import *
import pandas as pd
from numba import jit
from matplotlib import cm

@jit
# Potential function
def U(phi,I=1.01,gamma=1,T=4.615120516841259, tau=1):
    value = (I/gamma)*(1-np.exp(-gamma*T*phi/tau))
    return value
@jit
# Function to convert potential to 'phase' corresponding to Mirollo-Strogatz model
def U_inv(y,I=1.01,gamma=1,T=4.615120516841259,tau=1):
    value = (tau/(gamma*T))*np.log((1-(gamma*y/I))**(-1))
    return value
## Loading Inhibitory and Excitatory Edge topology from a 81 × 81 Matrix specifying 20 connections for each neuron 
# Import Sudoku matrices: (rows: pre-synaptic, columns: post-synaptic)

inhib_connect = np.loadtxt('Clueless_Sudoku_inhib.txt', dtype=int)
excit_connect = np.loadtxt('Clueless_Sudoku_excit.txt', dtype=int)

# Get tuples of corresponding connection indices for putting in Brian2's synapse connect() function
inhib_pre, inhib_post = inhib_connect.nonzero()
excit_pre, excit_post = excit_connect.nonzero()

AttributeError: module 'sympy' has no attribute 'functions'

In [ ]:
start_scope()
N = 81                          # No. of neurons
tau = 10*ms                     # Time constant of neuron
#A_osc = 0.2                     # Oscillation amplitude #?
freq = 25*Hz                    # Frequency of common oscillatory drive #?
transm_delay = pow(10,-5)*ms    # Delay between firing and reception of pulse by post-synaptic neuron
Ib_low = 0.58                   # Lower limit of bias current
Ib_upp = 0.64                   # Upper limit of bias current
Ib_val = [Ib_low + (Ib_upp-Ib_low)*( i/9.0 ) for i in range(9)]
order_Ib = np.array([9,1,7,5,2,3,4,6,8]) 
#Ib_array = np.zeros(N,dtype=float)


# Pulse strengths: inhibitory and excita tory
C_inh = -2.7*pow(10,-1)
C_exc = +2.8*pow(10,-6)

eqs = '''
dv/dt = (-v)/tau : 1
phase = (freq*t) % 1.0 : 1
'''
# Create a neuron group 'G' following the above equation
G = NeuronGroup(N, eqs, threshold='v>1', reset='v=0', method='euler')
G.v = 'rand()' #?

# Inhibitory synapses
S_inhib = Synapses(G,G, on_pre='v_post += C_inh', delay=transm_delay)
S_inhib.connect(i=inhib_pre, j=inhib_post)

# Excitatory synapses
S_excit = Synapses(G,G, on_pre='v_post += C_exc', delay=transm_delay)
S_excit.connect(i=excit_pre, j=excit_post)

P = PoissonInput(G, 'v', 1,10*Hz, weight=5)

# Record of potential for all neurons
potential = StateMonitor(G, 'v', record=True)
# Record of oscillatory drive (common to all neurons)
#osc = StateMonitor(G, "I", record=0)
# Record of all spikesfiring_time.t/ms
firing_time = SpikeMonitor(G, variables='phase')
run(10000*ms)

# Raster plot of firing
# Plot firing from 'index1' no. of iterations before end

figure(figsize=(25,10))
title('Exc = +'+str(C_exc) + ', '+ 'Inh = ' + str(C_inh), fontsize=16)
plot(firing_time.t/ms, firing_time.i,'.k',markersize=10)
xlabel('Time (ms)', fontsize=16)
ylabel('Neuron index', fontsize=16)
show()